# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [37]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages (0.24.2)


### Import Libraries

In [38]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from scipy import stats

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# magic word for producing visualizations in notebook
%matplotlib inline

ImportError: cannot import name '_deprecate_positional_args'

### List of Functions

In [ ]:
def convert_unknown_to_nan(df, keys_dict):
    """
    Replaces given keys from keys_dict to np.nan in df
    
    Input:
        df (DataFrame): Dataset for which keys to need to be converted nan 
        keys_dict: dictionary of column names with keys that need to be converted to nan

    """
    # Search for unknown keys in each column and replace

    for column_name in keys_dict:
        if column_name in df.columns:       
            keys_values = keys_dict[column_name]
            for key in keys_values:
                if key == 'X' or key == 'XX':
                    key = str(key)
                else:
                    key = int(key)
                
                df[column_name].replace(key, np.NaN, inplace=True)

#########################################################################################################

def drop_columns(df, missing_columns, threshold=0.3):
    """
    Columns with more than 30% missing values will be dropped.

    INPUT:
    - df: DataFrame to be cleaned
    - missing_columns: count of missing values per column
    - threshold: threshold as decision criteria for dropping (default 0.3)

    OUTPUT:
    - clean_df: DataFrame with a smaller percentage of missing values than threshold
    - drop_null_cols: List of columns that were dropped
    """
    
    column_missing_values_perc = missing_columns/len(df)                
    drop_null_cols = list(column_missing_values_perc[column_missing_values_perc>threshold].index)
    clean_df = df.drop(drop_null_cols, axis=1)
    
    return clean_df, drop_null_cols

#########################################################################################################

def drop_rows(df, threshold=0.3):
    """
    Rows with more than 30% missing values will be dropped.

    INPUT:
    - df: DataFrame to be cleaned
    - missing_rows: count of missing values per row
    - threshold: threshold as decision criteria for dropping (default 0.3)

    OUTPUT:
    - clean_df: DataFrame with a smaller percentage of missing values than threshold
    """
    missing_rows= df.isnull().sum(axis=1)
    clean_df = df[missing_rows / df.shape[1] <= threshold]
    
    return clean_df
    
#########################################################################################################    
    
def clean_data(df, unknown_values_dict, drop_cols, test_data=False):
    """
    Cleans the dataframe: 
    - Convert unknowns to NaN
    - Drop columns and rows with high amount of mising values and further unvaluable columns ('LNR')
    - Convert columns 'EINGEFUEGT_AM' and 'OST_WEST_KZ'
    
    Input:
        df: Dataset that needs to be cleaned up 
        unknown_values_dict: Dictionary containing column names with keys that need to be converted to nan
        drop_cols: list of columns to be dropped
        test_data (True/False): True if dataframe is mailout_test dataframe, here rows shouldn't be dropped

    Output:
        clean_df: Clean dataset
    """
    # Convert unknowns to NaN
    convert_unknown_to_nan(df, unknown_values_dict)
    
    # drop columns with more than 30% missing values
    clean_df = df.drop(drop_cols, axis=1)
    
    # drop columns with unique identifiers (here: "LNR")
    clean_df = clean_df.drop(['LNR'], axis=1)
    
    # drop rows with more than 30% missing values
    if test_data==False:
        clean_df = drop_rows(clean_df, threshold=0.3)
                             
    # Convert columns 'EINGEFUEGT_AM' and 'OST_WEST_KZ'
    clean_df['EINGEFUEGT_AM'] = pd.to_datetime(clean_df['EINGEFUEGT_AM'], format='%Y-%m-%d ').dt.year
    
    clean_df['OST_WEST_KZ'].replace('O', 0, inplace=True)
    clean_df['OST_WEST_KZ'].replace('W', 1, inplace=True)
    clean_df['OST_WEST_KZ'] = pd.to_numeric(clean_df['OST_WEST_KZ'], errors = 'coerce')
    

    return clean_df

#########################################################################################################

class Log1pTransformer(BaseEstimator, TransformerMixin):
    """
    Transforms skewed continuous data to log to get normal distributed data
    
    Input: 
    columns: List of columns to be log-transformed (not mandatory)
    
    Output:
    X_transformed: Dataframe with log-transformed data
    
    """
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        for column in self.columns:
            try:
                X_transformed[column] = np.log1p(X_transformed[column])
            except ValueError:
                X_transformed[column] = np.NaN
        return X_transformed
    
#########################################################################################################

# Custom Imputer function
class CustomImputer(TransformerMixin):
    def __init__(self, columns=None, strategy='mean'):
        self.cols = columns
        self.strategy = strategy
            
    def transform(self, df):
        X = df.copy()
        impute = SimpleImputer(strategy=self.strategy)
        
        if self.cols == None:
            self.cols = list(X.columns)
        for col in self.cols:
            if X[col].dtype == np.dtype('O') : 
                X[col].fillna(X[col].value_counts().index[0], inplace=True)
            else : X[col] = impute.fit_transform(X[[col]])
        return X
    
    def fit(self, *_):
        return self

##########################################################################################################


def int_to_str(column):
    """
    Custom function to convert integer to string  
    """
    return column.astype(str)

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

### Load the data and basic evaluation
- read in csv files `Udacity_AZDIAS_052018.csv` and `Udacity_CUSTOMERS_052018.csv`
- basic data evaluation: shape, head, info, describe, isnull

In [ ]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

In [ ]:
# basic data evaluation AZDIAS
print(azdias.shape)
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.head())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.info())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.describe())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.isnull().sum())

In [ ]:
# basic data evaluation CUSTOMERS
print(customers.shape)
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.head())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.info())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.describe())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.isnull().sum())

## Data Preprocessing Steps

- Explore features (Differences in columns)

- Check data types

- Explore features using `DIAS Attributes - Values 2017.xlsx`

- Missing Values 
    - Convert Unknown to NaN
    - Columns with NaN (distribution of % missing --> decide what to drop)
    - Rows with NaN (distribution of % missing --> decide what to drop)

- Datatypes: Classify attribute types (https://www.mygreatlearning.com/blog/types-of-data/)
    - Quantitative Attributes (Numerical)
        - Discrete / Continous data
    - Qualitative Attributes (Categorical)
    - Impute NaN for each data type?

- Encoding / Standardization (OneHotEncoder / StandardScaler)

### Explore Features 

In [ ]:
# differences in columns
azdias_cols = set(azdias.columns)
customers_cols = set(customers.columns)
customers_cols - azdias_cols


There are 3 columns in the customers data set not included in azdias data set: 'CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'.

### Check Data Types

In [ ]:
# check data types azdias
azdias.get_dtype_counts()

In [ ]:
# check data types customers
customers.get_dtype_counts()

In [ ]:
print('Categorical features in AZDIAS:\n', azdias.select_dtypes(['object']).head(5))
print('----------------------------------------------------------------------------------------------------------------------')
print('Categorical features in CUSTOMERS:\n', customers.select_dtypes(['object']).head(5))

There are 6 categorical features in the azdias data set but 8 categorical featrues in the customers data set. 

The feature information to these categorical features are the following:
- CAMEO_DEU_2015 / CAMEO_DEUG_2015 / CAMEO_INTL_2015: These describe the sociodemographic background of a person or household based on the CAMEO classification system.
- D19_LETZTER_KAUF_BRANCHE: Industry in which last purchase was made
- EINGEFUEGT_AM: Date where record was added 
- OST_WEST_KZ: Inidcates whether a house is located in West Germany / Federal Republik of Germany / FRG (W) or East Germany / German Democratic Republik / GDR (O)
- PRODUCT_GROUP: Product Group where the transactional activity has taken place
- CUSTOMER_GROUP: Classification of the customer as MULTI_BUYER or SINGLE_BUYER 

### Explore Features using DIAS Attributes - Values 2017.xlsx

In [ ]:
# explore features using DIAS Attributes - Values 2017.xlsx
attr_values = pd.read_excel('DIAS Attributes - Values 2017.xlsx', usecols='B:E', dtype='str')
attr_values.head()

There are several values in the features that have the meaning "unknown". These should be replaced by NaN in the dataset in order to perform a correct estimation and cleanup of missing values.
Among numerical characteristics, there are many whose values have natural, ordered categories. These need to be identified and classified as ordinal or categorical data.

In [ ]:
# identify values with meaning unknown
unknown_values = attr_values[attr_values["Meaning"].isin(["unknown","unknown / no main age detectable"])]
unknown_values.head()

### Missing Values
- Convert Unknown to NaN
- Columns with NaN (distribution of % missing --> decide what to drop)
- Rows with NaN (distribution of % missing --> decide what to drop)

In [ ]:
# Create dict of missing attribute-value combinations
unknown_values_dict = {}
for _, row in unknown_values.iterrows():
    key = row["Attribute"]
    unknown_values_dict[key] = row["Value"].split(", ")
    

In [ ]:
# Manually add missing codes from other/similar attributes 
unknown_values_dict["CAMEO_INTL_2015"] = ['XX']
unknown_values_dict["CAMEO_DEUG_2015"] = ['X','XX']
unknown_values_dict["CAMEO_DEU_2015"] =['XX']
unknown_values_dict["GEBURTSJAHR"] = ['0']


In [ ]:
# Number of missing values before converted to nan
azdias.isnull().sum().sort_values(ascending=False)

In [ ]:
# Split df into samples to reduce data amount (to prevent Jupyter Kernel from dying)
azdias_1 = azdias.iloc[:300000]
azdias_2 = azdias.iloc[300000:600000]
azdias_3 = azdias.iloc[600000:]

In [ ]:
# Convert unknowns to nan
convert_unknown_to_nan(azdias_1, unknown_values_dict)
#convert_unknown_to_nan(azdias_2, unknown_values_dict)
#convert_unknown_to_nan(azdias_3, unknown_values_dict)

# concat samples to one df
#frames = [azdias_1, azdias_2, azdias_3]
#azdias = pd.concat(frames)

# Trying to convert the whole azdias df the kernel of Jupyter died. Here just a sample is cleaned and used for the next steps. 

In [ ]:
# Number of missing values AFTER converted to nan
azdias_1.isnull().sum().sort_values(ascending=False)

In [ ]:
# selecting only columns with missing values
column_missing_values = azdias_1.isnull().sum().sort_values(ascending=False)
column_missing_values = column_missing_values[column_missing_values > 0]

print("There are {} columns with missing values.".format(len(column_missing_values)))

In [ ]:
# Top 100 columns with highest count of missing values
column_missing_values.head(100).plot.bar(figsize=(16,8))
plt.title('Columns and number of missing values')
plt.xlabel('Column')
plt.ylabel('Number of missing values')
plt.show()

In [ ]:
# Share of missing values
column_missing_values_perc = column_missing_values/len(azdias_1) * 100
column_missing_values_perc

In [ ]:
# Top 100 columns with highest share of missing values in %
column_missing_values_perc.head(100).plot.bar(figsize=(16,8))
plt.title('Top 100 missing columns')
plt.xlabel("Column")
plt.ylabel("Share of missing values in %")
plt.show

print("9 columns have 50% or more missing values. Most of the columns have less than 20% missing values.")

In [ ]:
# Plot distribution of missing values
plt.figure(figsize=(16,8))
plt.hist(column_missing_values_perc, bins=np.linspace(5,100))
plt.title('Distribution of missing data in columns')
plt.xticks(np.arange(0, 100, 5))
plt.ylabel('# of columns')
plt.xlabel('% of missing data')

print(column_missing_values_perc.describe())
print('----------------------------------------------------------------------------------------------------------------------')
print("On average, columns have a missing share of {}%.".format(round(column_missing_values_perc.mean(),1)))
print("97% of columns have a missing share of {}% or less.".format(round(column_missing_values_perc.quantile(0.97),1)))


In [ ]:
# Drop columns with >30% missing values
azdias_dropped_cols, drop_null_cols = drop_columns(azdias_1, column_missing_values, 0.3)

print("{} columns have more than 30% missing values and will be will be dropped: {}".format(len(drop_null_cols),drop_null_cols))

In [ ]:
# Missing values in Rows:
missing_rows = azdias_1.isnull().sum(axis=1)

plt.figure(figsize=(16,8))
plt.hist(missing_rows, bins=np.linspace(min(missing_rows), max(missing_rows)+1))
plt.title('Distribution of missing values in rows');
plt.xticks(np.arange(min(missing_rows), max(missing_rows)+1, 10))
plt.ylabel('# of rows')
plt.xlabel('# of missing values')


In [ ]:
azdias_dropped_rows = drop_rows(azdias_1, 0.3)

print("In total {}% of rows were dropped having >30% missing values.".format(round(100-100*azdias_dropped_rows.shape[0]/len(azdias_1),0)))

### Clean Data

In [ ]:
azdias_sample = azdias.iloc[:10000]
clean_azdias_sample = clean_data(azdias_sample, unknown_values_dict, drop_null_cols, test_data=False)

print("{} rows are dropped and {} columns.".format(azdias_sample.shape[0]-clean_azdias_sample.shape[0], azdias_sample.shape[1]-clean_azdias_sample.shape[1]))
clean_azdias_sample.head()

# Trying to clean the whole azdias df the kernel of Jupyter died. Here just a sample is cleaned and used for the next steps. 

In [ ]:
customers_sample = customers.iloc[:10000]
clean_customers_sample = clean_data(customers_sample.drop(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis=1), unknown_values_dict, drop_null_cols, test_data=False)

print("{} rows are dropped and {} columns.".format(customers_sample.shape[0]-clean_customers_sample.shape[0], customers_sample.shape[1]-clean_customers_sample.shape[1]))
clean_customers_sample.head()

### Classify attribute types

In [ ]:
# total number of numerical columns:
num_cols = clean_azdias_sample.select_dtypes(['float64', 'int64']).columns

binary_cols = []
for col in num_cols:
    n_unique = clean_azdias_sample[col].dropna().nunique()
    if n_unique == 2:
        binary_cols.append(col)
        
discrete_cols = list(clean_azdias_sample.select_dtypes(['int64']).columns)
continuous_cols = list(set(num_cols) - set(discrete_cols) - set(binary_cols))

# total number of categorical columns:
cat_cols = list(clean_azdias_sample.select_dtypes(['object']).columns)

print("Number of numerical columns: {} \nNumber of categorical columns: {}\n".format(len(num_cols), len(cat_cols)))

print("List of continuous columns: {} \n\nList of binary columns: {} \n\nList of discrete columns: {} \n\nList of categorical columns: {}".format(continuous_cols, binary_cols, discrete_cols, cat_cols))

### Impute, encode and standardize features
- Quantitative Attributes (Numerical)
    - Discrete: Missing values of discrete features will be imputed based on the median value and standardized using StandardScaler()
    - Continous: All continuous features will be imputed based on median. Because there are some discrete features in the list due to their float datatype, the decimal should stay 0 after imputation. Skewed features will be log-transformed first and imputed based on the median value to avoid bias. All continuous features will be standardized with StandardScaler()
    - Binary: Missing values will be imputed based on the most common value.
- Qualitative Attributes (Categorical)
    - Categorical features will be one-hot encoded and then imputed on the most common value.

In [ ]:
# Discrete features:
discrete_pipeline = Pipeline([
    ('discrete_impute', SimpleImputer(strategy='median')),
    ('discrete_scale', StandardScaler())
])

# Continuous features:
# Check for skewness
skewed_cols = []
for col in continuous_cols:
    try:
        skewness = skew(clean_azdias_sample[col].dropna())
        if abs(skewness) > 1:
                        skewed_cols.append(col)
    except:
         pass

#print(skewed_cols)

# Skewed continuous features:

skewed_pipeline = Pipeline([
    ('skewed_transform', Log1pTransformer(skewed_cols)),
    ('skewed_impute', SimpleImputer(strategy='median')),
    ('skewed_scale', StandardScaler())
])

# Normal disctributed continuous features:
normal_pipeline = Pipeline([
    ('normal_impute', SimpleImputer(strategy='median')),
    ('normal_scale', StandardScaler())
])

# Binary features:
binary_pipeline = Pipeline([(
    'binary_impute', SimpleImputer(strategy='most_frequent'))
])


# Categorical features:
categorical_pipeline = Pipeline([
    ('binary_impute', SimpleImputer(strategy='most_frequent')),
    ('string_conversion', FunctionTransformer(int_to_str, validate=False)),
    ('encoding', OneHotEncoder(sparse=False))
])

### Transform Features

In [ ]:
discrete_pipeline.fit_transform(clean_azdias_sample[discrete_cols])

skewed_pipeline.fit_transform(clean_azdias_sample[skewed_cols])

normal_cols = list(set(continuous_cols)-set(skewed_cols))
normal_pipeline.fit_transform(clean_azdias_sample[normal_cols])

binary_pipeline.fit_transform(clean_azdias_sample[binary_cols])

categorical_pipeline.fit_transform(clean_azdias_sample[cat_cols])

In [ ]:
discrete_pipeline.fit_transform(clean_customers_sample[discrete_cols])

skewed_pipeline.fit_transform(clean_customers_sample[skewed_cols])

normal_cols = list(set(continuous_cols)-set(skewed_cols))
normal_pipeline.fit_transform(clean_customers_sample[normal_cols])

binary_pipeline.fit_transform(clean_customers_sample[binary_cols])

categorical_pipeline.fit_transform(clean_customers_sample[cat_cols])

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

- Apply PCA
- Plot cumulative explained variance (y) per number of components (x) -> decide how many components are necessary to explain the variance (variance treshhold somewhere between 80-95%)
- Apply PCA with reduced number of components
- Analyze the top three components and the columns with highest weight in each of the components --> gives idea about columns with highest importance/influence
- Apply K-Means
- Plot elbow curve to determine number of clusters
- Apply PCA and K-Means with same params to customer data set
- pickle cluster results
- compare both datasets in terms of cluster-sizes: plot population_count and customers_count per cluster (in %)
- Analyze overrepresentated and underrepresentated clusters from customer data set

In [ ]:
# Cumulative variance per numer of components (AZDIAS)

pca = PCA()
pca_full_azdias = PCA.fit_transform(clean_azdias_sample)

plt.plot(np.cumsum(PCA.explained_variance_ratio_))
plt.title('Explained variance by principal components (AZDIAS)')
plt.xlabel('Number of principle components')
plt.ylabel('Explained variance')

plt.show()



## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')